In [ ]:
import tensorflow as tf
from tensorflow import keras
import os
import numpy as np
import pandas as pd
import random

In [ ]:
def einlesen(): 

    #FALLS
    FOL=[]
    FKL=[]
    BSC=[]
    SDL=[]
    
    #ADL
    CHU=[]
    SCH=[]
    STU=[]
    STN=[]
    CSI=[]
    CSO=[]
    JOG=[]
    JUM=[]
    SIT=[]
    STD=[]
    WAL=[]

    #Path to MobiAct_Dataset_v2
    pfad="\\MobiAct_Dataset_v2"
    #FALLS
   
    #FOL    
    FOL=[pd.read_csv(pfad+"\Annotated Data\FOL\FOL_{0}_{1}_annotated.csv".format(i,a),usecols=["rel_time","acc_x","acc_y","acc_z","gyro_x","gyro_y","gyro_z"])
    for a in range(1,4)
        for i in range(1,68)
            if os.path.exists(pfad+"\Annotated Data\FOL\FOL_{0}_{1}_annotated.csv".format(i,a))
            ]
    return FOL,FKL,BSC,SDL,CHU,SCH,STU,STN,CSI,CSO,JOG,JUM,SIT,STD,WAL

In [ ]:
#1 = fall 0 = ADL - Ad labels
def label(data, is_fall):
       # Falls (1)
  if is_fall == 1:
      for i in range(len(data)):
          data[i]["Label"]=1  
       #ADLs (0)        
  else: 
      for i in range(len(data)):
          data[i]["Label"]=0

  return data

In [ ]:
#returns a data_vector
def create_data_vector(list): 
    data_vector=[]
    for item in list: 
        for i in range(len(item)): 
            data_vector.append(item[i])
    
    #Shuffle array
    random.shuffle(data_vector)
    return data_vector

In [ ]:
#Position: 0 1 DataFrames Label -- Matrix Create
def create_data_matrix(data_vector):
   data_matrix=([],[])
   for i in range(len(data_vector)):
       data_matrix[0].append(data_vector[i][["acc_x","acc_y","acc_z","gyro_x","gyro_y","gyro_z"]])
       data_matrix[1].append(data_vector[i].at[0,"Label"]) # Only the label is taken out of the entire column
       
   return data_matrix

In [ ]:
def data_convert():
   test=[],[]
   train=[],[]
   test = data_matrix[0][:400],data_matrix[1][:400] # Test and traindata split
   train = data_matrix[0][400:],data_matrix[1][400:] 
   
   print("Split Successful! Number of training data:{0} and test data:{1} ".format(len(train[0]),len(test[0]))) 

   train_data=[]
   labels=[]
   test_data=[]
   test_labels=[]

   for i in range(len(train[0])):
       train_data.append(train[0][i].values)


   for i in range(len(train[1])):
       labels.append(train[1][i])
   labels=np.array(labels)
   for i in range(len(test[0])):
       test_data.append(test[0][i].values) 
   for i in range(len(test[1])): 
       test_labels.append(test[1][i])
   test_labels=np.array(test_labels)
   return train_data,labels,test_data, test_labels

In [ ]:
# construct the model
def train_model(train_data,labels,test_data, test_labels):

   model = keras.Sequential([     
   keras.layers.Flatten(), 
   keras.layers.Masking(mask_value=0.0),
   keras.layers.Dense(109, activation=tf.nn.softsign), 
   keras.layers.Dense(50, activation=tf.nn.softmax),
   ]) 
   optimizer=keras.optimizers.Nadam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.003)
   model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
   
   #We had to bring every array to the same length
   train_data=keras.preprocessing.sequence.pad_sequences(train_data, maxlen=None, dtype='int32', padding='pre', truncating='pre', value=0.0)
   train_data=np.array(train_data)
   test_data=keras.preprocessing.sequence.pad_sequences(test_data, maxlen=train_data.shape[1], dtype='int32', padding='pre', truncating='pre', value=0.0)
   test_data=np.array(test_data) 
   history=model.fit(train_data,labels,validation_split=0.15,batch_size=100, epochs=5500 ) 
   
   
   model.summary()  
   print(test_labels.shape,labels.shape)
   print(test_data.shape,train_data.shape)
   # Test Model
   test_loss, test_acc = model.evaluate(test_data, test_labels) 
   print('Test accuracy:', test_acc) 
   print('Test loss:', test_loss)
   model.save('simple_mlp5500.pb') 

In [ ]:
#Run
print("GPU Available: ", tf.test.is_gpu_available()) #GPU rendering speeds up the process
FOL,FKL,BSC,SDL,CHU,SCH,STU,STN,CSI,CSO,JOG,JUM,SIT,STD,WAL=einlesen()
#Label all the data

FOL=label(FOL,1)   
FKL=label(FKL,1)
BSC=label(BSC,1)
SDL=label(SDL,1)
CHU=label(CHU,0)
SCH=label(SCH,0)
STU=label(STU,0)
STN=label(STN,0)
CSI=label(CSI,0)
CSO=label(CSO,0)
JOG=label(JOG,0)
JUM=label(JUM,0)
SIT=label(SIT,0)
STD=label(STD,0)
WAL=label(WAL,0)

data_vector=create_data_vector([FOL,FKL,BSC,SDL,CHU,SCH,STU,STN,CSI,CSO,JOG,JUM,SIT,STD,WAL])
data_matrix=create_data_matrix(data_vector)

train_data, labels,test_data, test_labels=data_convert()
train_model(train_data,labels, test_data, test_labels)

